In [15]:
#imports

import pandas as pd
import hvplot
from pathlib import Path
import hvplot.pandas
import numpy as np
import cartopy.feature
import cartopy.crs as ccrs
from fbprophet import Prophet
import requests
from bs4 import BeautifulSoup


Our Predictions vs. Oddsmakers

In [16]:
#Our predicted Beijing Total Medal Count
beijing_total_medal_count_forecast_df = pd.read_csv(Path('Resources/beijing_total_medal_count_forecast.csv'))
forcast_df = beijing_total_medal_count_forecast_df
forcast_df = forcast_df.replace('United States', 'USA')
forcast_df

,Country,Gold,Silver,Bronze,Total
0,Germany,13.304938,12.423865,8.468517,34.200475
1,USA,8.737703,8.930442,8.167370,25.873304
2,Norway,9.542529,8.494180,7.470627,25.586041
3,Canada,8.636452,7.711165,6.090819,22.464903
4,ROC,3.723646,7.287529,11.142462,22.269231
5,Austria,4.930853,5.561746,7.326002,17.864528
6,South Korea,5.162205,7.557807,3.423116,16.131148
7,Netherlands,5.998734,4.097720,5.313305,15.422468
8,Switzerland,4.610455,3.585948,3.956117,12.068417
9,Italy,3.282677,3.278728,5.432943,11.994356


In [17]:
# Actual Final Medal Count
final_medal_count_2022 = pd.read_html('https://olympics.com/beijing-2022/olympic-games/en/results/all-sports/medal-standings.htm') 
final_medal_count_2022 = final_medal_count_2022[0]
final_medal_count_2022 = final_medal_count_2022.rename(
    columns={'NOC': 'Country', 'Unnamed: 2': 'Gold', 'Unnamed: 3': 'Silver','Unnamed: 4': 'Bronze'})
final_medal_count_2022 = final_medal_count_2022.drop(columns={'OrderbyTotal', 'NOCCode'})
final_medal_count_2022 = final_medal_count_2022.replace('People\'s Republic of China', 'China')
final_medal_count_2022 = final_medal_count_2022.replace('United States of America', 'USA')
final_medal_count_2022 = final_medal_count_2022.replace('Republic of Korea', 'South Korea')
final_medal_count_2022

,Order,Country,Gold,Silver,Bronze,Total
0,1,Norway,16,8,13,37
1,2,Germany,12,10,5,27
2,3,China,9,4,2,15
3,4,USA,8,10,7,25
4,5,Sweden,8,5,5,18
5,6,Netherlands,8,5,4,17
6,7,Austria,7,7,4,18
7,8,Switzerland,7,2,5,14
8,9,ROC,6,12,14,32
9,10,France,5,7,2,14


In [18]:
final_medal_count_2022.hvplot.bar(
    title ='Total Number of Gold Medals - Beijing Winter Olympic - 2022',
    color='gold',
    x='Country',
    y='Gold',
    rot=90
)

:Bars   [Country]   (Gold)

In [19]:
final_medal_count_2022.hvplot.bar(
    title ='Total Number of Silver Medals - Beijing Winter Olympic - 2022',
    color='silver',
    x='Country',
    y='Silver',
    rot=90
)

:Bars   [Country]   (Silver)

In [20]:
final_medal_count_2022.hvplot.bar(
    title ='Total Number of Bronze Medals - Beijing Winter Olympic - 2022',
    color='#CD7F32',
    x='Country',
    y='Bronze',
    rot=90
)

:Bars   [Country]   (Bronze)

In [21]:
final_medal_count_2022.hvplot.bar(
    title ='Total Number of Medals with Projected Totals - Beijing Winter Olympic - 2022',
    width=900,
    height=400,
    color='Blue',
    label='Total Actual Medals',
    x='Country',
    y='Total',
    rot=90
) * forcast_df.hvplot.scatter(
    x='Country', 
    y='Total',
    color='orange',
    label='Predicted Medals',
    rot=90)

:Overlay
   .Bars.Total_Actual_Medals :Bars   [Country]   (Total)
   .Scatter.Predicted_Medals :Scatter   [Country]   (Total)

In [8]:
#Calculate the fractional forecast medal total - total predicted medals
# predicted medals/fractional

medal_total = 327
forcast_df['Fractional'] = forcast_df.apply(lambda row: medal_total - row.Total, axis = 1)
forcast_df


,Country,Gold,Silver,Bronze,Total,Fractional
0,Germany,13.304938,12.423865,8.468517,34.200475,292.799525
1,USA,8.737703,8.930442,8.167370,25.873304,301.126696
2,Norway,9.542529,8.494180,7.470627,25.586041,301.413959
3,Canada,8.636452,7.711165,6.090819,22.464903,304.535097
4,ROC,3.723646,7.287529,11.142462,22.269231,304.730769
5,Austria,4.930853,5.561746,7.326002,17.864528,309.135472
6,South Korea,5.162205,7.557807,3.423116,16.131148,310.868852
7,Netherlands,5.998734,4.097720,5.313305,15.422468,311.577532
8,Switzerland,4.610455,3.585948,3.956117,12.068417,314.931583
9,Italy,3.282677,3.278728,5.432943,11.994356,315.005644


In [9]:
# Calculate Decimal value from fractional total/fractional 
# Add 1 to the fractional representation e.g. a fractional value of 3/1 = (3/1) + 1 = 4.

forcast_df['Decimal'] = forcast_df.apply(lambda row: (row.Total/row.Fractional) + 1, axis = 1)
forcast_df


,Country,Gold,Silver,Bronze,Total,Fractional,Decimal
0,Germany,13.304938,12.423865,8.468517,34.200475,292.799525,1.116805
1,USA,8.737703,8.930442,8.167370,25.873304,301.126696,1.085922
2,Norway,9.542529,8.494180,7.470627,25.586041,301.413959,1.084887
3,Canada,8.636452,7.711165,6.090819,22.464903,304.535097,1.073768
4,ROC,3.723646,7.287529,11.142462,22.269231,304.730769,1.073078
5,Austria,4.930853,5.561746,7.326002,17.864528,309.135472,1.057789
6,South Korea,5.162205,7.557807,3.423116,16.131148,310.868852,1.051891
7,Netherlands,5.998734,4.097720,5.313305,15.422468,311.577532,1.049498
8,Switzerland,4.610455,3.585948,3.956117,12.068417,314.931583,1.038321
9,Italy,3.282677,3.278728,5.432943,11.994356,315.005644,1.038077


In [10]:
# Calculate American odds from decimal odds 

#Positive american odds - (The decimal value minus one) multiplied by 100 e.g. a decimal value of 3 = (3 - 1) * 100 = 200.
#Negative american odds - Negative 100 divided by (the decimal odds minus one) e.g. a decimal value of 1.2 = -100 / (1.2 - 1) = -500.

forcast_df['American'] = forcast_df['Decimal'].apply(lambda x: -100/(x - 1) if x > 0 else (x-1)*100)
forcast_df = forcast_df.replace('United States', 'USA')
forcast_df


,Country,Gold,Silver,Bronze,Total,Fractional,Decimal,American
0,Germany,13.304938,12.423865,8.468517,34.200475,292.799525,1.116805,-856.127071
1,USA,8.737703,8.930442,8.167370,25.873304,301.126696,1.085922,-1163.850956
2,Norway,9.542529,8.494180,7.470627,25.586041,301.413959,1.084887,-1178.040632
3,Canada,8.636452,7.711165,6.090819,22.464903,304.535097,1.073768,-1355.603881
4,ROC,3.723646,7.287529,11.142462,22.269231,304.730769,1.073078,-1368.393767
5,Austria,4.930853,5.561746,7.326002,17.864528,309.135472,1.057789,-1730.442987
6,South Korea,5.162205,7.557807,3.423116,16.131148,310.868852,1.051891,-1927.134089
7,Netherlands,5.998734,4.097720,5.313305,15.422468,311.577532,1.049498,-2020.283213
8,Switzerland,4.610455,3.585948,3.956117,12.068417,314.931583,1.038321,-2609.551717
9,Italy,3.282677,3.278728,5.432943,11.994356,315.005644,1.038077,-2626.282261


In [11]:
# moneyline odds from oddsshark.com

medal_odds_2022 = pd.read_html('https://www.oddsshark.com/olympics/medal-odds/') 
odds_shark_df = medal_odds_2022[0]
odds_shark_df = odds_shark_df.replace('Russian Olympic Committee', 'ROC')
odds_shark_forcast_df = odds_shark_df
odds_shark_forcast_df = odds_shark_forcast_df.rename(columns={'Odds': 'American'})
odds_shark_forcast_df

,Country,American
0,Norway,-800
1,ROC,900
2,Germany,1000
3,Canada,1400
4,USA,1400
5,Netherlands,8000
6,Sweden,15000
7,Austria,25000
8,China,25000
9,France,50000


In [12]:
# Convert Americann odds to decimal odds
odds_shark_forcast_df['Decimal'] = odds_shark_forcast_df['American'].apply(lambda x: x/100 + 1 if x >= 0 else 100/abs(x)+1)
odds_shark_forcast_df


,Country,American,Decimal
0,Norway,-800,1.125
1,ROC,900,10.000
2,Germany,1000,11.000
3,Canada,1400,15.000
4,USA,1400,15.000
5,Netherlands,8000,81.000
6,Sweden,15000,151.000
7,Austria,25000,251.000
8,China,25000,251.000
9,France,50000,501.000


In [13]:
# Convert American to Fractional
# Positive odds - The american odds divided by 100 e.g. american odds of 300 = 300/100 = 3/1.
# Negative odds - Minus 100 divided by the american odds e.g. american odds of -300 = -100/-300 = 1/3.

odds_shark_forcast_df['Fractional'] = odds_shark_forcast_df['American'].apply(lambda x: x/100 if x >= 0 else -100/x)
odds_shark_forcast_df


,Country,American,Decimal,Fractional
0,Norway,-800,1.125,0.125
1,ROC,900,10.000,9.000
2,Germany,1000,11.000,10.000
3,Canada,1400,15.000,14.000
4,USA,1400,15.000,14.000
5,Netherlands,8000,81.000,80.000
6,Sweden,15000,151.000,150.000
7,Austria,25000,251.000,250.000
8,China,25000,251.000,250.000
9,France,50000,501.000,500.000



Betting What-ifs - If I had bet $100 on $? Country total medals in 1/1/2022 I would have won $? 
    

In [14]:
#Scatter Plot Comparison of Predictive Data Odds Shark vs. Project Team Predictions
forcast_df.hvplot.scatter(
    width=1000,
    height=400,
    title='Odds Shark Predictions vs. Team Historical Predictions',
    label='Team Predictions',
    x='Country', 
    y='Decimal',
    ylabel='Eurpean Odds',
    color='purple',
    rot=90) * odds_shark_forcast_df.hvplot.scatter(
    width=800,
    x='Country', 
    y='Decimal',
    ylabel='Odds',
    label='Odds Shark Predictions', rot=90)


:Overlay
   .Scatter.Team_Predictions       :Scatter   [Country]   (Decimal)
   .Scatter.Odds_Shark_Predictions :Scatter   [Country]   (Decimal)